In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
import pandas as pd
import statsmodels.api as sm
import math

# 1. Bisection


One of the most common algorithms for numerical root-finding is *bisection*.

To understand the idea, recall the well-known game where:

- Player A thinks of a secret number between 1 and 100  
- Player B asks if it’s less than 50  
  
  - If yes, B asks if it’s less than 25  
  - If no, B asks if it’s less than 75  
  

And so on.

This is bisection, a relative of [binary search](https://en.wikipedia.org/wiki/Binary_search_algorithm). It works for all sufficiently well behaved increasing continuous functions with $ f(a) < 0 < f(b) $. 

Write an implementation of the bisection algorith, `bisect(f, lower, upper, tol)` which, given a function `f`, a lower bound `lower` and an upper bound `upper` finds the point `x` where `f(x) = 0`. The parameter `tol` is a numerical tolerance, you should stop once your step size is smaller than `tol`.


Use it to minimize the function:

$$
f(x) = \sin(4 (x - 1/4)) + x + x^{20} - 1 \tag{2}
$$

in python: `lambda x: np.sin(4 * (x - 1/4)) + x + x**20 - 1`

The value where f(x) = 0 should be around `0.408`

In [ ]:
def bisect(f, lower, upper, tol):

    temp_upper = upper
    temp_lower = lower

    for step in range(tol):

        x = ((temp_upper - temp_lower) / 2 ) + temp_lower
        result = f(x)

        if result == 0:
            return x

        if result > 0:
            temp_upper = temp_upper - ((temp_upper - temp_lower) / 2 )

        else:
            temp_lower = temp_lower + ((temp_upper - temp_lower) / 2 ) 
            
    return x

f= lambda x: np.sin(4 * (x - 1/4)) + x + x**20 - 1

print(bisect(f, 0, 100, 20))
# It locks in on 0.4 around 10 steps and improves from there.

# 1.2 (stretch) Recursive Bisect

Write a recursive version of the bisection algorithm

# 2.1 Movies Regression

Write the best linear regression model you can on the [Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset?select=ratings.csv) to predict the profitability of a movie (revenue - budget). Maintain the interpretability of the model.

Few notes:

1. Clean your data! Movies where the budget or revenue are invalid should be thrown out

2. Be creative with feature engineering. You can include processing to one-hot encode the type of movie, etc.

3. The model should be useful for someone **who is thinking about making a movie**. So features like the popularity can't be used. You could, however, use the ratings to figure out if making "good" or "oscar bait" movies is a profitable strategy.

In [ ]:
# Import the data
meta_df = pd.read_csv(r'C:\Users\David\Documents\code\Module 3\3-5-optimization\archive\movies_metadata.csv')

#Clean a few key columns
meta_df.revenue = pd.to_numeric(meta_df.revenue, errors='coerce')
meta_df.budget = pd.to_numeric(meta_df.budget, errors='coerce')
meta_df.runtime = pd.to_numeric(meta_df.runtime, errors='coerce')

#Extract the genre strings
meta_df['genre'] = meta_df.genres.str.findall(pat='([A-Z]\w*)')

#More cleaning
meta_df = meta_df.dropna(subset=['budget', 'revenue', 'runtime', 'genres'])
meta_df = meta_df[(meta_df.budget != 0)]

#Create some new variables
meta_df.release_date = pd.to_datetime(meta_df.release_date)
meta_df['month'] = meta_df.release_date.dt.strftime("%B")
meta_df['profit'] = meta_df['revenue'] - meta_df['budget']

#Make a new, cleaner dataframe for regression
df = pd.DataFrame(meta_df[['profit', 'runtime', 'budget']])
df = df.join(pd.get_dummies(pd.get_dummies(meta_df.genre.apply(pd.Series).stack(), drop_first=True).sum(level=0)))
df = df.join(pd.get_dummies(meta_df.month, drop_first=True))

#Final clean
df = df.replace(-np.inf, np.nan)
df = df.dropna()

#Run the model
y = df.profit
x = df.drop(['profit'], axis=1)

X = sm.add_constant(x)
my_model = sm.OLS(y,X).fit()

my_model.summary()

## Explained:

I realize this is far from a good, reliable regression. I kept it because in the chaos, there are some serious takeaways. Within genres and months, we see some with strong coefficients and p-values, which denotes a viable relationship to profit. Others have unreliable p-values and small coefficients, which is worth noting for its non-relationship (it does not increase or reduce profit)

# 2.2 Movies Manual Regression

Use your `X` and `y` matrix from 2.1 to calculate the linear regression yourself using the normal equation $(X^T X)^{-1}X^Ty$.

Verify that the coefficients are the same.

In [64]:
from numpy.linalg import inv

normal_matrix = inv(X.T.dot(X)).dot(X.T).dot(y)

coeffs = my_model.params #This is a catch-all sum of the coefficient, what happens when you move between points

print('First attempt:', normal_matrix)
print('First attempt:',coeffs)

#This is how I started, and clearly the coefficients do not match. Instead, let's simplify the regression.

y_ = df.profit
x_ = df[['runtime', 'budget']]

X_ = sm.add_constant(x_)
my_model_ = sm.OLS(y_,X_).fit()

normal_matrix_ = inv(X_.T.dot(X_)).dot(X_.T).dot(y_)

coeffs_ = my_model_.params #This is a catch-all sum of the coefficient, what happens when you move between points

print('------------------------------------------')

print('Second attempt:',normal_matrix_)
print('Second attempt:',coeffs_)

#Now it's working. Clearly my poor regression is not replicable manually. I will continue the workshop with this (simple) regression instead.

First attempt: [-1.25913468e+07  3.49901396e+04  1.94049783e+00]
First attempt: const         -2.019091e+07
runtime        1.657177e+05
budget         1.826289e+00
Adventure      1.080808e+07
Animation      2.043897e+07
Comedy        -4.087082e+06
Crime         -2.670676e+06
Documentary    5.356431e+06
Drama         -6.806587e+06
Family         5.264381e+06
Fantasy        1.774105e+06
Fiction       -1.835344e+06
Foreign       -1.706955e+06
History       -2.298770e+07
Horror         8.683341e+06
Movie         -4.547748e+06
Music          2.499181e+06
Mystery       -2.280183e+06
Romance        4.859021e+06
Science       -1.835344e+06
TV            -4.547748e+06
Thriller      -8.570603e+06
War           -9.346423e+06
Western       -2.675549e+07
August        -6.376887e+06
December       5.074125e+06
February      -5.517921e+06
January       -5.146609e+06
July           2.163225e+06
June           2.021745e+07
March         -5.547655e+06
May            1.147140e+07
November       5.715161e

# 2.3 Movies gradient descent regression

Use your `X` and `y` matrix from 2.1 to calculate the linear regression yourself using **gradient descent**. 

Hint: use `scipy.optimize` and remember we're finding the $\beta$ that minimizes the squared loss function of linear regression: $f(\beta) = (\beta X - y)^2$. This will look like part 3 of this lecture.

Verify your coefficients are similar to the ones in 2.1 and 2.2. They won't necessarily be exactly the same, but should be roughly similar.

In [104]:
y = np.array(df.profit)
x = np.column_stack((X.const, np.column_stack((X.runtime, X.budget))))
from scipy.optimize import minimize

def gradientloss(betas, y, x):
    result = 0
    for i in range(0, len(y)):
        xb = np.dot(x[i], betas)
        loss = (xb - y[i])**2
        result += loss
    return result

bhat = np.zeros(len(x[0]))

gradient_est = minimize(gradientloss, bhat, args=(y,x), method='nelder-mead')

print('Gradient descent attempt:',gradient_est['x'] * np.std(y))
print('True coefficients:',coeffs_)

#It would be a stretch to say this is perfect but it seems to be approaching the correct coefficients.

Gradient descent attempt: [-1.49254181e+08  2.16794411e+07  2.08288458e+08]
True coefficients: const     -1.259135e+07
runtime    3.499014e+04
budget     1.940498e+00
dtype: float64
